# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Trabajo práctico entregable - Grupo 22 - Parte 2

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

El dataset a utilizar es la encuesta Sysarmy del año 2022 versión 2, en formato csv, la cual es una una encuesta personal y voluntaria que busca relevar información sobre salarios y condiciones de trabajo de programadores, que se realiza anualmente. Se analizarán sólo los datos provenientes de Argentina. 

Se utilizará un dataset que ya ha tenido un pretratamiento: 
* Se eliminó el encabezado de la encuesta original.
* Se renombraron las columnas.

Este dataset está disponible en internet, desde donde lo usaremos.

In [2]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)
total_ans = len(df) # cantidad de respuestas en tel dataset

---
# Ejercicio 1 - Estimación

## Creación del DataFrame a utilizar

Vamos a basarnos en el mismo DataFrame que utilizamos para la parte 1 del entregable. Sus características son:
- Se eliminaron los valores faltantes.
- Se renombraron las columnas.
- Tener un sueldo bruto estrictamente mayor al sueldo neto.
- Tener un sueldo neto entre un Salario Mínimo Vital y Móvil (SMVM) en Abril de 2022 ($ 38.940) y 2 millones de pesos.
- Tener el 85 % central de la distribución.
- Recategorizar el género dentro de varón cis, mujer cis y diversidades.

In [13]:
print(f'Cantidad inicial de filas: {total_ans}.')

# Selección de columnas
relevant_columns = ["salary_monthly_BRUTO",
                    "salary_monthly_NETO",
                    "profile_gender"]

df1 = df[relevant_columns]

# Eliminación de missing values
df1 = df1.dropna(subset=relevant_columns)

# Modificación de etiquetas a usar
df1.rename(columns = {"salary_monthly_BRUTO":'bruto', 
                      "salary_monthly_NETO":'neto',
                      "profile_gender":'genero'}
                      , inplace = True)

# Bruto mayor al neto
df1 = df1[df1["bruto"]>df1["neto"]]

# Entre un SMVM y 2 millones
SMVM = 38940
df1 = df1[df1["neto"]>=SMVM]
df1 = df1[df1["neto"]<=2*1e6]

# Dejar de lado la variable bruto
relevant_columns = ["neto", "genero"]
df1 = df1[relevant_columns]

# Tomar el 85% central de la distribución
k = 15
percentile_inf = df1["neto"].quantile(k * 0.5 / 100)
percentile_sup = df1["neto"].quantile((100 - k * 0.5) / 100)

df1 = df1[df1["neto"] > percentile_inf]
df1 = df1[df1["neto"] < percentile_sup]

# Recategorizar la variable género
df1.loc[:,'genero'] = df1.genero.replace(
    {'Varón Cis': 'varon_cis',
     'Varón cis': 'varon_cis',
     'Mujer': 'mujer_cis',
     'Mujer Cis': 'mujer_cis',
     'Mujer cis': 'mujer_cis',
     'Femenino': 'mujer_cis',
     'mujer': 'mujer_cis',
     'Mujer':'mujer_cis',
     'Queer':'diversidades',
     'Varón Trans':'diversidades',
     'No binarie':'diversidades',
     'Mujer Trans':'diversidades',
     'Fluido':'diversidades',
     'Bigénero':'diversidades',
     'Gay':'diversidades'
    })

print(f'Quedan {len(df1)} filas.')

display(df1[['genero', 'neto']].groupby('genero').describe())

Cantidad inicial de filas: 5358.
Quedan 3481 filas.


neto                                                                              
               count           mean           std      min       25%       50%       75%       max
genero                                                                                            
diversidades    53.0  204218.270755  73934.335271  92000.0  144000.0  190000.0  253892.0  450000.0
mujer_cis      583.0  203723.703310  81025.162448  91983.0  140000.0  195000.0  245000.0  497000.0
varon_cis     2845.0  227844.365803  88618.445676  91600.0  158944.0  220000.0  280000.0  499886.0

## Definiciones para la estadística

nivel de significancia, agrupacion, IC, etc

In [15]:
# Nivel de significancia
alpha = 0.05

# Agrupación por ser o no varón cis
man = df1.genero == 'varon_cis'

Varones = df1[man].neto
nVarones = df1[~man].neto

## Consigna

Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?